# ADC Simulation
Version: 12.2020

Mit dieser Simulation sollen grundsätzlich RunUp und RunDown Funktionen

# Library laden


In [1]:
import SimADC_V1 as adc
import pandas as pd
import plotly.graph_objects as px

import plotly.io as pio
pio.renderers.default = 'iframe'


## Simulation starten


In [2]:
print('Simulation startet')

# https://tinyurl.com/y9j2ls63

# 100k bei 0,1% = 100R bei 0,01% = 10R
# 2.5042
integrator = adc.Integrator()
integrator.Vin = -5.0

integrator.VrefN = -7.0448
integrator.VrefP =  7.0448
integrator.ChargingInjection = 10.0 # pC 

integrator.R_VrefN = 100.0  # [kOhm]
integrator.R_VrefP = 100.00  # [kOhm]
integrator.R_Vin =   200.0  # [kOhm]

integrator.C = 2.2 # in nF

cpu = adc.CPU()
cpu.WaveForm = 10

# CPU Ticks = 0.1 us => 10 Mhz
# Anzahl der Rampen ( PLC 60Hz = 16,6ms  PLC 50Hz = 20 ms)
# Schalterwechsel mit einem Impuls von 200 ns ( 1/ 16)
cpu.RunUpCycles = 3280   # (6250 * 3,2 us = 20 ms = 50 Hz)
cpu.RunUpCnt    = 320 # => 20us

# Auflösung: 6250 * 200 = 1.250.000
# Bei Vin +/-10V = 20V / Aufslösung = 10 uV (theoretisch) 


cpu.Connect( integrator) # Kontrolle für den Integrator an die CPU übergeben
cpu.ResidueADC = adc.ResidueADC(cpu, 12, 3.0) # Reste ADC mit n bit erzeugen

# Kalibration Array 
# Parameter: Vadc Vin gemessen ADC
# Parameter: Vtrue Vin real
calibration_points = [
    (-5.051280630094539, -5.0),  # -Vref
    (-0.0012881896324126111, 0.0),            # 0 V
    (5.048704250829712 ,   5.0)    # +Vref
]

# Initialisiere die Kalibrierung
#cpu.Calibration = adc.ADCCalibration(calibration_points)



cpu.Reset()
#cpu.AutoZero()
cpu.RunUp()
cpu.RunDown()
cpu.CalcVin()
#cpu.CalcVinX()







Simulation startet


## Diagramm erstellen
Vout am Integrator

Beipiele und Dokumention finden sie auf der folgenden Seite:
https://plotly.com/python/<br>
https://plotly.com/python/bar-charts/#basic-bar-chart-with-plotlygraphobjects

In [3]:
In
x_vals = [ float(i.Cnt) for i in integrator.RunUpCells]
y_vals = [ float(i.Value) for i in integrator.RunUpCells]

#print(x_vals)


# neues Diagramm erstellen mit Titel
fig = px.Figure()


# Linie anzeigen
fig.add_trace( px.Scatter( x=x_vals, y=y_vals , name="Vout" )  )

fig.update_layout(
   title_text='Multi-Slope (Runup)',
    yaxis_title="V",
    xaxis_title="Clock",
    
    font=dict(
        family="Courier New, monospace",
        size=12,
        color="#7f7f7f"
    )
)



fig.show()




In [4]:
print("Vin: ", integrator.Vin, " V")
print("Vref: ", integrator.VrefP, " V")
print("Vout", integrator.Vout, " V")
print("RunUp-Time: ", cpu.RunUpTime() , " ms")
print("RunDown-Count :", cpu.RunDownCnt *  0.1 , "us (Ticks)")
print("RunDown-Count :", cpu.RunDownCnt )
print("Vin berechnet: ", cpu.Vin , " V")
print("Vout-Before : ", cpu.Vout_Before , " V")
print("SIGN  : ", cpu.RunDown_Sign , " bit")



print("calc Vin V1  : ", cpu.V1 , " V")
print("calc Vin V2  : ", cpu.V2 , " V")
print("CntN :", cpu.CntN )
print("CntP :", cpu.CntP )
print("VAutoZero : mV", cpu.VAutoZero  ) 
print("Resolution uV:", cpu.Resolution * 1000.0 * 1000.0)


Vin:  -5.0  V
Vref:  7.0448  V
Vout 1.0666618181819798  V
RunUp-Time:  104.96  ms
RunDown-Count : 0.0 us (Ticks)
RunDown-Count : 0
Vin berechnet:  -5.322436402439026  V
Vout-Before :  0.0  V
SIGN  :  0  bit
calc Vin V1  :  5.317964878048781  V
calc Vin V2  :  0.004471524390244582  V
CntN : 2259
CntP : 1021
VAutoZero : mV 0.0
Resolution uV: 0.0


In [5]:
# Ausgabe 100 Werten

cpu = adc.CPU()

integrator.VrefN = -7.0448
integrator.VrefP = 7.0448
cpu.WaveForm = 0

integrator.R_Vin   = 50.0 # k
integrator.R_VrefN = 50.000   # k
integrator.R_VrefP = 50.000  # k
cpu.RunUpCycles = 6250 # Anzahl der Rampen ( PLC 50Hz = 120ms )
cpu.RunUpCnt    = 30   # 10 Mhz (200 * 0.1us) = 20us
integrator.C = 1.50 #pF


cpu.Connect(integrator )
#cpu.ResidueADC = adc.ResidueADC(cpu, 16) # Reste ADC mit n bit erzeugen


print(f" N : {cpu.RunUpCycles} Cycle : {cpu.RunUpCnt/10} us VrefN : {integrator.VrefN} V  VrefP : {integrator.VrefP} V  Rn : {integrator.R_VrefN} k Rp : {integrator.R_VrefP} k Rin : {integrator.R_Vin} k")
print("")
integrator.Vin = 0.0
for a in range(0,100+1):
    
    cpu.Reset()
    cpu.RunUp()
    cpu.RunDown()
    cpu.CalcVin() # Berechnet den Wert der CPU - Vin 
    

    
    delta = (integrator.Vin - cpu.Vin) 
        
    if( delta < 0 ): 
        vor="(-)" 
    else: 
        vor="(+)"
    
#    print(f" Vin : {integrator.Vin:2.4f} CntN : {cpu.CntN } CntP :  {cpu.CntP} RunDownCnt : {cpu.RunDownCnt:4d} Sign : {cpu.RunDown_Sign} Vor: {vor} Vin berechnet: {cpu.Vin:2.4f} Delta: {abs(delta):2.4f} Factor:{((delta)/cpu.Vin):2.4f} Korrigiert:{korrektur:2.4f}")
    print(f" Vin: {integrator.Vin:2.4f} CntN: {cpu.CntN } CntP:  {cpu.CntP} v1: {cpu.V1:2.4f} V2: {cpu.V2:2.4f} Vor: {vor} Vinc: {cpu.Vin:2.8f} Delta: {delta*1000:2.2f}mV")
    integrator.Vin = integrator.Vin + 0.05
    
    

 N : 6250 Cycle : 3.0 us VrefN : -7.0448 V  VrefP : 7.0448 V  Rn : 50.0 k Rp : 50.0 k Rin : 50.0 k

 Vin: 0.0000 CntN: 3125 CntP:  3125 v1: 0.0000 V2: 0.0000 Vor: (-) Vinc: 0.00003757 Delta: -0.04mV
 Vin: 0.0500 CntN: 3103 CntP:  3147 v1: 0.0000 V2: 0.0000 Vor: (+) Vinc: 0.04914452 Delta: 0.86mV
 Vin: 0.1000 CntN: 3082 CntP:  3168 v1: 0.0000 V2: 0.0000 Vor: (+) Vinc: 0.09738732 Delta: 2.61mV
 Vin: 0.1500 CntN: 3060 CntP:  3190 v1: 0.0000 V2: 0.0000 Vor: (+) Vinc: 0.14656941 Delta: 3.43mV
 Vin: 0.2000 CntN: 3038 CntP:  3212 v1: 0.0000 V2: 0.0000 Vor: (+) Vinc: 0.19567636 Delta: 4.32mV
 Vin: 0.2500 CntN: 3017 CntP:  3233 v1: 0.0000 V2: 0.0000 Vor: (+) Vinc: 0.24395673 Delta: 6.04mV
 Vin: 0.3000 CntN: 2995 CntP:  3255 v1: 0.0000 V2: 0.0000 Vor: (+) Vinc: 0.29310125 Delta: 6.90mV
 Vin: 0.3500 CntN: 2973 CntP:  3277 v1: 0.0000 V2: 0.0000 Vor: (+) Vinc: 0.34220820 Delta: 7.79mV
 Vin: 0.4000 CntN: 2952 CntP:  3298 v1: 0.0000 V2: 0.0000 Vor: (+) Vinc: 0.39048857 Delta: 9.51mV
 Vin: 0.4500 CntN

In [6]:
# Kalibrierung

x1 = 0.0
y1 = 0.0
x2 = 0.0
y3=  0.0

cpu = adc.CPU()
integrator.VrefN = -7.0443
integrator.VrefP = 7.04476

integrator.R_Vin   = 60.0   # k
integrator.R_VrefN = 30.0   # k
integrator.R_VrefP = 30.1  # k
cpu.RunUpCycles = 6000   # Anzahl der Rampen ( PLC 60Hz = 16,6ms )
cpu.RunUpCnt    = 20    # 10 Mhz (200 * 0.1us) = 20us
integrator.C = 1.0 #pF


cpu.Connect(integrator )
cpu.ResidueADC = adc.ResidueADC(cpu, 16) # Reste ADC mit n bit erzeugen

integrator.Vin = 0.0    
cpu.Reset()
cpu.RunUp()
cpu.RunDown()
cpu.CalcVin() # Berechnet den Wert der CPU - Vin 

x1 = integrator.Vin
y1 = cpu.Vin

integrator.Vin = 5.000    
cpu.Reset()
cpu.RunUp()
cpu.RunDown()
cpu.CalcVin() # Berechnet den Wert der CPU - Vin 

x2 = integrator.Vin
y2 = cpu.Vin

# y1 = m x x1 + b
# x1 = (y1-b) / m

m = (y2-y1)/(x2-x1)

b = y1 - (m*x1)


integrator.Vin = 3.000    
cpu.Reset()
cpu.RunUp()
cpu.RunDown()
cpu.CalcVin() # Berechnet den Wert der CPU - Vin 



y3 = cpu.Vin
x3 = (y3-b) / m

print(f" x1: {x1:2.4f}  y1: {y1:2.4f}")
print(f" x2: {x2:2.4f}  y2: {y2:2.4f}")
print(f" x3: {x3:2.4f}  y3: {y3:2.4f}")





 x1: 0.0000  y1: -0.2067
 x2: 5.0000  y2: 20.2351
 x3: 4.1195  y3: 16.6351
